In [0]:
### Ingestion file Production company

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

production_company_schema = StructType([
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True)
])

production_company = spark\
    .read\
    .schema(production_company_schema)\
    .csv(f"{bronze_folder_path}/{v_file_date}/production_company")

production_company_df = production_company\
    .withColumnRenamed("companyId", "company_id")\
    .withColumnRenamed("companyName", "company_name")\
    .withColumn("ingestion_date", current_timestamp())\
    .withColumn("environment", lit("PROD"))\
    .withColumn("file_date", lit(v_file_date))

# overwrite_partition(production_company_df, "movie_silver", "production_company", "file_date")

merge_condition = "tgt.company_id = src.company_id AND tgt.file_date = src.file_date"
merge_delta_lake(production_company_df, "movie_silver", "production_company", silver_folder_path, merge_condition, "file_date")

# production_company_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.production_company")


In [0]:
%sql
select file_date, count(1) 
from movie_silver.production_company
group by file_date;

In [0]:
%sql
select *
from movie_silver.production_company;

In [0]:
dbutils.notebook.exit("Success")